Sort_asian_face02.pyで作成したchapt-06-model1.pthを用いて、  
UIをもったアプリケーションを作成してAIを実際に使用する。カメラからの映像をキャプチャし、画面にメッセージを表示するGUIアプリを作成する  
アプリにはキャプチャ画像とメッセージの他にボタンを配置して写っている人物の顔を登録できるようにする。  
そしてカメラからの映像に含まれる顔が登録されている顔の人物と同じかどうかの判定を行う。

## 主な使い方と説明
このコードを実行すると顔を認識するためにカメラが立ち上がり、windowに自分の顔が映し出される。それに沿って「この顔を登録する」というボタンを押す。  
これによってface_vactorにその顔のベクトルが格納され、その後、登録した顔であると判定された場合は、登録ユーザである旨を、登録した顔でないと判定された場合は、未登録であるという旨を表示。  
しかし、この顔のベクトルはこのコードの実行を終了してしまうとデータが消えてしまうということが欠点となる。  
### 今後やるべきこと
・webアプリ化するなどして使用しやすくすることと  
・顔のデータを毎回消えないようにデータサーバなどに保存できるようにしておくこと(AmazonS3などかな？)  
・学生証番号をセットで入力してもらうことで顔と学生証番号を結びつけてpostやmoodleと連携するなどして広告の表示方法を考える。

## 必要なパッケージのインポート

In [1]:
import numpy as np
from PIL import Image, ImageTk
import cv2
from time import time,sleep
from tkinter import Tk, NW, TOP, Frame, Canvas, Label, Button, StringVar
import threading

import torch
from torch import nn

from sklearn.metrics.pairwise import cosine_similarity

from facenet_pytorch import MTCNN, InceptionResnetV1



## モデルの作成  
変数には同一人物かどうかの判定を行うための閾値があり、「THREASHHOLD」という名前で作成する。  
閾値の値はモデルの評価を行った際に表示される、最も良いスコアを出した際の閾値を使用する。作成したモデルには「chapt-06-model1.pyh」のファイルにある重みを読み込む

In [2]:
#gpuを使用するかどうか
use_device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
#アプリケーションが実行中かどうか
is_run = True
#同一人物と判断するための閾値
threashhold = 0.58 #転移学習済みのモデルでの最善値(Sort_asian_face_02.pyより)
#保存しておいたモデルを読み込む
model = InceptionResnetV1(pretrained='vggface2')
model.load_state_dict(torch.load('/Users/oonoharuki/PyTorchで始めるAI開発/learningmodel/chapt06/chapt06-model1.pth',map_location=torch.device(use_device)))

#モデルを推論用にする
model.eval()
model.to(use_device)

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

## モデルの実行
OpenCVの画像を引数にとってモデルの実行結果を返す関数を作成する。  
ニューラルネットワークの実行は与えられた顔画像のTensorを、1バッチのサイズに変形してモデルに入れるもの

In [3]:
def detect_one(face_img):
    #バッチサイズ１のtensorにする
    face_img = torch.unsqueeze(face_img,0)
    #gpuを使用する場合、gpuに転送する
    face_img = face_img.to(use_device)
    #ニューラルネットワークをを実行する
    batch_result = model(face_img)
    #バッチ内から結果を所得
    result = batch_result.detach().cpu().numpy()
    result = result[0]
    return result

## GUIの作成
顔を新しく登録するボタン「self.button」として作成する。  
顔検出の結果を入れる「self.face_detect」と登録されている顔のベクトルを入れる「self.face_vector」変数を用意する。  



In [4]:
class MyFrame(Frame):
    def __init__(self, parent, **params):
        Frame.__init__(self, parent, params)
        #OpenCVのカメラキャプチャを準備
        self.cap = cv2.VideoCapture(0)
        #画像の表示場所を作成
        self.image = Image.new('RGB',(320,240),(0,0,0))
        self.imgtk = ImageTk.PhotoImage(self.image)
        self.canvas = Canvas(self, width=320, height=240, bg='black')
        self.canvas.place(x=20,y=20)
        self.canvas.create_image(0,0,image=self.imgtk,anchor=NW,tag='i')
        #認識結果を表示場所を作成する
        self.message = StringVar()
        self.message.set('')
        self.label = Label(self, textvariable=self.message)
        self.label.place(x=360,y=20)
        #顔の登録ボタン
        self.button = Button(self,text="この顔を登録する", command=self.add_face)
        self.button.place(x=360,y=200)
        #顔検出の結果
        self.face_detect = None
        #登録されている顔のベクトル
        self.face_vector = []
        #顔検出
        self.mtcnn = MTCNN()
        
    def add_face(self):
        #顔を認証済みリストに追加する
        if self.face_detect is not None:#顔検出の結果があれば
            #画像をニューラルネットワークに入れて実行する
            vector = detect_one(self.face_detect)
            #結果を追加する
            self.face_vector.append(vector)
    
    def updateFrame(self):
        #OpenCVでカメラからキャプチャ
        ret, frame = self.cap.read()
        frame = cv2.resize(frame, (320,240))
        #顔の検出
        try:
            #顔の位置検出
            rgb = frame[:,:,::-1]#OpenCVのBGRをRBGにする
            batch_boxes, _ = self.mtcnn.detect(rgb)
            #顔を切り出す
            self.face_detect = self.mtcnn.extract(rgb,batch_boxes,None)
        except Exception as e :
            self.face_detect =None
        #赤色の枠線を書く
        if self.face_detect is not None and batch_boxes is not None:
            left, top, right, bottom = tuple(batch_boxes[0])
            frame = cv2.rectangle(frame,(int(left),int(top)),(int(right),int(bottom)),(0,0,255),3)
            #openCVのBGRをRBGにする
            frame = frame[:,:,::-1]
            #画像を表示する
            self.image = Image.fromarray(frame)
            self.imgtk = ImageTk.PhotoImage(self.image)
            self.canvas.itemconfigure(tagOrId='i' , image=self.imgtk)
            #0.1秒後に再び更新
            self.after(100, self.updateFrame)

## メッセージの表示
スレッド内の無限ループで「frame.face_detect」から顔検出の結果を受け取り「detect_one」関数でベクトルデータ化する。  
そして「frame.face_vector」変数にある登録されている顔のベクトルデータとcos類似度を求める。  
cos類似度は「threashhead」変数に設定しておいた閾値以上であれば、同一人物とみなす。そして、1つでも同一人物と見做された場合は「登録ユーザです」、そうでない場合は「未登録ユーザです」とメッセージを表示する。  

In [5]:
#ニューラルネットワークを実行する
def detect(frame):
    global is_run
    n_pred = 0
    s_time = time()
    lpf_window = [] #ローパスフィルタのウィンドウ
    with torch.no_grad():
        #アプリケーションの実行中は無限ループ
        while is_run:
            #キャプチャ速度に合わせて最大fpsを調整する
            a_time = time()
            #顔検出の結果と登録さてれている顔がある場合のみ実行する
            if frame.face_detect is not None and len(frame.face_vector) > 0:
                result = detect_one(frame.face_detect)
                #メッセージの更新
                n_pred += 1
                #経過時間
                fps = n_pred / (time() - s_time)
                #結果から表示メッセージを作成する
                sim = cosine_similarity([result],frame.face_vector)
                usr = (sim > threashhold).any()
                predict = '''
                登録ユーザです。
                本日の予定は〇〇です。
                ''' if usr else '未登録です「この顔を登録する」を押してください。'
                #表示メッセージ
                message = f'''認識結果：{predict}
                認識実行回数:{n_pred}
                認識速度:{fps}fps'''
                #UI上に表示
                frame.message.set(message)
                #0.1秒以下だったらその分待つ
                deltime = (time() - a_time)
                if deltime < 0.1:
                    sleep(0.1-deltime)

## アプリケーションの起動
GUIのウィンドウとスレッドを立ち上げて、アプリケーションを起動する

In [6]:
#画面いっぱいにウィンドウを作成する
win = Tk()
win.geometry('800x360')
frame = MyFrame(win, width=800, height=360)
frame.pack(side=TOP)
win.after_idle(frame.updateFrame)#起動後にupdateFrameを呼び出す

detection = threading.Thread(target =detect, args=(frame,))
detection.start()

win.mainloop()#処理を開始

#スレッドの終了を待つ
is_run = False
detection.join()